-------Train results:
-----Mini-batch size 256
---Fixed: EPOCHS = 1000,LEARNING_RATE = 1e-6

THETA = 1e-5
In epoch 1000, loss is 5.291985059220746
In epoch 1000 accuracy is 0.346075

THETA = 1e-4
In epoch 1000, loss is 5.292198365836736
In epoch 1000 accuracy is 0.348175

THETA = 1e-3
In epoch 1000, loss is 5.296268629829621
In epoch 1000 accuracy is 0.34815

THETA = 1e-2
In epoch 1000, loss is 5.297345092785761
In epoch 1000 accuracy is 0.347775



---Fixed: EPOCHS = 1000,LEARNING_RATE = 8e-5
THETA = 1e-3
In epoch 1000, loss is 6.837051169043007
In epoch 1000 accuracy is 0.413175



---Fixed: EPOCHS = 1000,LEARNING_RATE = 1e-5

THETA = 1e-5
In epoch 1000, loss is 4.473005846222188
In epoch 1000 accuracy is 0.387375

THETA = 1e-4
In epoch 1000, loss is 4.4734887011987485
In epoch 1000 accuracy is 0.3884

THETA = 1e-3
In epoch 1000, loss is 4.4808710837865116
In epoch 1000 accuracy is 0.385575

THETA = 1e-2
In epoch 1000, loss is 4.501979782009769
In epoch 1000 accuracy is 0.38755



---Fixed: EPOCHS = 1000,LEARNING_RATE = 1e-4

THETA = 1e-5
In epoch 1000, loss is 4.195538879880822
In epoch 1000 accuracy is 0.401625

THETA = 1e-4
In epoch 1000, loss is 4.118187725967621
In epoch 1000 accuracy is 0.4106

THETA = 1e-3
In epoch 1000, loss is 4.187295139050325
In epoch 1000 accuracy is 0.40025

THETA = 1e-2
In epoch 1000, loss is 4.246982001690002
In epoch 1000 accuracy is 0.41245



---Fixed: EPOCHS = 1000,LEARNING_RATE = 1e-3

Too much unstable
THETA = 1e-5
In epoch 1000, loss is 8.297607196745993
In epoch 1000 accuracy is 0.329675

---Fixed: EPOCHS = 1000,LEARNING_RATE = 7e-5
THETA = 1e-4
In epoch 1000, loss is 5.174179245491815
In epoch 1000 accuracy is 0.412975

-----Mini-batch size 512
---Fixed: EPOCHS = 1000,LEARNING_RATE = 1e-4

THETA = 1e-5
In epoch 1000, loss is 4.077958433569045
In epoch 1000 accuracy is 0.40925

THETA = 1e-2
In epoch 1000, loss is 4.62524982252704
In epoch 1000 accuracy is 0.393

-----Mini-batch size 128
---Fixed: EPOCHS = 1000,LEARNING_RATE = 5e-5

THETA = 1e-4
In epoch 1000, loss is 4.328775130110178
In epoch 1000 accuracy is 0.393875

-------Validation Results:
-----Mini-batch size 256
THETA = 1e-4
LEARNING_RATE = 7e-5
EPOCHS = 1000
Loss is 5.305431801173743,Accuracy is 0.3993

THETA = 1e-3
LEARNING_RATE = 8e-5
EPOCHS = 1000
Loss is 6.92635428403829,Accuracy is 0.4024

-------Test Results:
-----Mini-batch size 256
THETA = 1e-3
LEARNING_RATE = 8e-5
EPOCHS = 1000
Loss is 6.95635111972989,Accuracy is 0.3975


In [ ]:
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt

THETA = 1e-3
LEARNING_RATE = 8e-5
EPOCHS = 1000
MARGIN = 1


In [ ]:
def TrainMain():
    """
    Main function for training and getting weight matrix 
    """
    # load CIFAR-10 train, validation data
    xTrain, yTrain, xValid, yValid = load_train_data("CIFAR-10-train")

    # load CIFAR-10 test data
    xTest, yTest = load_test_data("CIFAR-10-test")

    # initialize a random weight matrix for CIFAR-10
    W = np.random.rand(10, 3073)*0.001

    pltEpoch=range(EPOCHS)
    pltAccuracy=[]
    pltLoss=[]

    # train the weights, epoch loop
    for epoch in range(EPOCHS):
        loss, accuracy = SVM_loss(xTrain, yTrain, W, theta=THETA)
        print(
            f'In epoch {epoch+1}, loss is {loss}\nIn epoch {epoch+1} accuracy is {accuracy}')

        pltAccuracy.append(accuracy)
        pltLoss.append(loss)

        # mini-batch sample 256 examples
        data_batch, yData_batch = sample_training_data(xTrain, yTrain, 256)

        weights_grad = evaluate_gradient(data_batch, yData_batch, W)

        # perform parameter update
        W -= LEARNING_RATE * weights_grad

    plotResults(pltEpoch,pltAccuracy,pltLoss)
    np.savetxt('W.txt', W)

In [ ]:
def ValidationMain():
    """
    Main function for validating the Validation set
    """
    # load CIFAR-10 train, validation data
    xTrain, yTrain, xValid, yValid = load_train_data("CIFAR-10-train")

    # load CIFAR-10 test data
    xTest, yTest = load_test_data("CIFAR-10-test")

    # get W from Trainmain() use it to validate
    W = np.loadtxt('W.txt')

    # calculate loss and accuracy
    loss, accuracy = SVM_loss(xValid, yValid, W, theta=THETA)
    print(
        f'Loss is {loss},Accuracy is {accuracy}')


In [ ]:
def TestMain():
    """
    Main function for testing the Test set
    """
    # load CIFAR-10 train, validation data
    xTrain, yTrain, xValid, yValid = load_train_data("CIFAR-10-train")

    # load CIFAR-10 test data
    xTest, yTest = load_test_data("CIFAR-10-test")

    # get W from Trainmain() use it to test
    W = np.loadtxt('W.txt')

    # calculate loss and accuracy
    loss, accuracy = SVM_loss(xTest, yTest, W, theta=THETA)
    print(
        f'Loss is {loss},Accuracy is {accuracy}')

In [ ]:
def unpickle(file):
    # only used in load_train_data() and load_test_data() to read pickle file
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


In [ ]:
def load_train_data(filename):
    """
    Load CIFAR-10 train data
    xTrain combined image arrays
    yTrain is the vector of correct labels for each image
    """

    # get train data 1-6 append them to a list
    temp_xTrain = []
    temp_yTrain = []
    for file in os.listdir(filename):
        data = unpickle(os.path.join(filename, file))
        temp_xTrain.append(data[b'data'])
        temp_yTrain.append(data[b'labels'])

    # make one list from the list of lists train data is now ready
    xTrain = np.concatenate(temp_xTrain)
    yTrain = np.concatenate(temp_yTrain)

    # change datatype to float
    xTrain = xTrain.astype('float64')

    # normalize data ???
    for id, im in enumerate(xTrain):
        normIm = im/255
        xTrain[id] = normIm

    # center data ???
    avIm = xTrain.mean(axis=0, keepdims=True)
    for id, im in enumerate(xTrain):
        normIm = im-avIm
        xTrain[id] = normIm

    # append bias to images arrays
    xTrain = np.c_[xTrain, np.ones(50000).T]

    # assign 1/5 of the images as validation dataset
    xValid = xTrain[:len(xTrain)//5]
    xTrain = xTrain[len(xTrain)//5:]

    yValid = yTrain[:len(yTrain)//5]
    yTrain = yTrain[len(yTrain)//5:]

    # transpose so xTrain is 3073x40000 yTrain is 40000x1 xValid is 3073x10000 yValid is 10000x1
    xTrain, yTrain, xValid, yValid = xTrain.T, yTrain.T, xValid.T, yValid.T

    return (xTrain, yTrain, xValid, yValid)


In [ ]:
def load_test_data(filename):
    """
    Load CIFAR-10 test data
    """
    data = unpickle(os.path.join(filename, os.listdir(filename)[0]))
    xTest = (data[b'data'])
    yTest = (data[b'labels'])
    yTest = np.array(yTest)

    # change datatype to float
    xTest = xTest.astype('float64')

    # normalize data ???
    for id, im in enumerate(xTest):
        normIm = im/255
        xTest[id] = normIm

    # center data ???
    avIm = xTest.mean(axis=0, keepdims=True)
    for id, im in enumerate(xTest):
        normIm = im-avIm
        xTest[id] = normIm

    # append bias to images arrays
    xTest = np.c_[xTest, np.ones(10000).T]

    # transpose so that xTest is 3073x10000 yTest is 10000
    xTest, yTest = xTest.T, yTest.T
    return (xTest, yTest)


In [ ]:
def SVM_loss(X, y, W, theta):
    """
    Multiclass Support Vector Machine Loss

    x is matrix of flattened image vectors with appended biases
    y is the integer valued matrix representing correct labels for images
    W is the weight matrix
    theta is the hyperparameter and usually determined by cross-validation

    We set up SVM loss so that for each image,
    its score of correct class is higher than every other
    incorrect class by a fixed margin.

    Multiclass SVM loss is formalized as:

    L = data_loss + theta*regularization_penalty

    wherein 
    data_loss =  (1/numOfTrainingData) *   sum    (max(0, (s_j - s_yi + margin)))
                                         (j != yi)
    (we assign margin value 1 and reduce number of hyperparameters
    because theta and margin are inverse proportional)

    regularization penalty = sum (sum ((W_k,l)**2))
                             (k)   (l)    
    """
    # weight times images
    score_matrix = W@X

    # get the correct class score for every image
    yi_scores = score_matrix[y, np.arange(score_matrix.shape[1])]
    yi_scores = np.matrix(yi_scores)

    # margins equals to below part, to ignore when s_j == s_yi we overwrite them by assigning 0s
    #    sum    (max(0, (s_j - s_yi + margin)))
    # (j != yi)
    margins = np.maximum(0, score_matrix - yi_scores + MARGIN)
    margins[y, np.arange(X.shape[1])] = 0

    # data loss and regularization part
    data_loss = np.mean(np.sum(margins, axis=0))
    regularization_penalty = np.sum(np.square(W))

    # total loss
    SVM_loss_score = data_loss + theta*regularization_penalty

    # find class where the score is the highest
    yPredict = np.argmax(score_matrix, axis=0)

    # calculate accuracy
    accuracy = np.mean(yPredict == y)

    return (SVM_loss_score, accuracy)


In [ ]:
def sample_training_data(trainData, yTrainData, num):
    """
    Mini-batch 
    num is the sample size
    """
    randIndices = np.random.choice(trainData.shape[1], num, replace=False)
    return (trainData[:, randIndices], yTrainData[randIndices])


In [ ]:
def evaluate_gradient(data_batch, yData_batch, W):
  """
  Gradient of SVM loss function
  data_batch consists random images selected from training set 
  yData_batch consists correct labels for each image
  """
  num_train=data_batch.shape[1]
  num_classes=W.shape[0]
  weight_grads=np.zeros(W.shape)

  for imgId in range(num_train):
      scores = W@data_batch[:,imgId]
      correct_class_score = scores[yData_batch[imgId]]
      for clsid in range(num_classes):
        margin = scores[clsid] - correct_class_score + MARGIN
        if margin > 0:
          if clsid == yData_batch[imgId]:
            weight_grads[clsid] -= data_batch[:,imgId] 
          else:
            weight_grads[clsid] += data_batch[:,imgId]
  return weight_grads

In [ ]:
def plotResults(pltEpoch,pltAccuracy,pltLoss):
    """
    Plot accuracy-epoch and loss-epoch
    """
    plt.figure(figsize=(18,10))
    plt.subplot(1,2,1)
    plt.plot(pltEpoch,pltAccuracy)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs Epoch')

    plt.subplot(1,2,2)
    plt.plot(pltEpoch,pltLoss)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss vs Epoch')

In [ ]:
if __name__ == "__main__":
    #TrainMain()
    #ValidationMain()
    #TestMain()